In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3,4,5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3,4,5


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ganime.data.base import load_dataset
from ganime.model.p2p.p2p_v3 import P2P
from ganime.visualization.videos import display_videos
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist", "../../data", batch_size=128)

2022-05-04 08:04:56.253261: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 08:04:57.232872: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14264 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:61:00.0, compute capability: 8.6
2022-05-04 08:04:57.233622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14264 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:81:00.0, compute capability: 8.6
2022-05-04 08:04:57.234283: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

In [5]:
strategy = tf.distribute.MirroredStrategy()

In [6]:
with strategy.scope():
    model = P2P(g_dim=128, z_dim=10)

In [7]:
model.encoder.summary(line_length=200)

Model: "encoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
input_4 (InputLayer)                                                                      [(None, 2, 64, 64, 1)]                                                          0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
time_distributed_10 (TimeDistributed)                                                     (None, 2, 32, 32, 64)                                                           1088     

In [8]:
model.decoder.summary(line_length=200)

Model: "decoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
input_5 (InputLayer)                                                                      [(None, 20, 10)]                                                                0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
dense_9 (Dense)                                                                           (None, 20, 512)                                                                 5632     

In [80]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
history = model.fit(train_ds, epochs=100)

2022-05-04 10:53:45.950659: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_UINT8
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 20
        }
        dim {
          size: 64
        }
        dim {
          size: 64
        }
        dim {
          size: 1
        }
      }
    }
  }
}



Epoch 1/100
71/71 [==============================] - 17s 120ms/step - loss: 55.2300 - reconstruction_loss: 52.3075 - kl_loss: 179.1536
Epoch 2/100
71/71 [==============================] - 9s 122ms/step - loss: 50.4696 - reconstruction_loss: 51.0568 - kl_loss: 179.0592
Epoch 3/100
71/71 [==============================] - 9s 122ms/step - loss: 50.8627 - reconstruction_loss: 50.9988 - kl_loss: 179.6728
Epoch 4/100
71/71 [==============================] - 9s 121ms/step - loss: 51.2713 - reconstruction_loss: 51.1281 - kl_loss: 179.7577
Epoch 5/100
71/71 [==============================] - 9s 122ms/step - loss: 51.1313 - reconstruction_loss: 51.2135 - kl_loss: 178.9938
Epoch 6/100
71/71 [==============================] - 9s 121ms/step - loss: 51.4214 - reconstruction_loss: 51.2046 - kl_loss: 179.0080
Epoch 7/100
71/71 [==============================] - 9s 122ms/step - loss: 51.3337 - reconstruction_loss: 51.3456 - kl_loss: 179.9149
Epoch 8/100
71/71 [==============================] - 9s 122ms

In [81]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)

In [82]:
generated = model(x[0])

In [83]:
display_videos(generated, n_rows=1, n_cols=5)

In [84]:
display_videos(x[1], n_rows=1, n_cols=5)

In [85]:
test_ds_iterator = test_ds.as_numpy_iterator()
x = next(test_ds_iterator)

In [86]:
generated = model(x[0])

In [87]:
display_videos(generated, n_rows=1, n_cols=5)

In [88]:
display_videos(x[1], n_rows=1, n_cols=5)

In [ ]:
model.save("B-VAE_LSTM_Longer")